# Check M1M3 and TMA configuration settings
Craig Lage 19-Jan-24

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker


# Subroutines to get the data

In [ ]:
async def getGGRGSettings(start):
    ggrg_components = ['useAccelerationForces', 'useBalanceForces', \
                   'useVelocityForces', 'triggerBoosterValves']
    ggrg_start = start - TimeDelta(1.0, format = 'jd')
    ggrg_end = start + TimeDelta(10.0, format = 'sec')
    ggrg = await client.select_time_series(\
    'lsst.sal.MTM1M3.logevent_slewControllerSettings', \
    ggrg_components, ggrg_start, ggrg_end)
    result = ''
    for component in ggrg_components:
        if ggrg[component].iloc[-1]:
            result += 'G'
        else:
            result += 'R'
    return result

async def getTMASettings(start):
    tma_components = ['maxMoveVelocity','maxMoveAcceleration','maxMoveJerk']
    tma_axes = ['azimuth', 'elevation']
    tma_start = start - TimeDelta(1.0, format = 'jd')
    tma_end = start + TimeDelta(10.0, format = 'sec')
    result = []
    for axis in tma_axes:
        this_axis_settings = []
        tma = await client.select_time_series(\
        f'lsst.sal.MTMount.logevent_{axis}ControllerSettings', \
        tma_components, tma_start, tma_end)
        for component in tma_components:
            this_axis_settings.append(tma[component].values[-1])
        result.append(this_axis_settings)
    return result

# Now check it on the run from 04-Jan-24

## No longer finds this old data

In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()
dayObs = 20240124
blockNum = 178        
events = eventMaker.getEvents(dayObs)
 # The lines below are from the logs
jerks = [[20,10],[10,5],[5,2.5],[1.0,0.5]]
settings = ["GGRG", "GRRG", "GGRR", "GRRR"]
startSeqNum = [196,204,212,227,240,248,256,264,274,289,297,312,322,336,344,379]

counter = 0
for [azJerk,elJerk] in jerks:
    commanded_az_settings = [4.0, 4.0, azJerk]
    commanded_el_settings = [2.0, 2.0, elJerk]
    for setting in settings:
        seqNum = startSeqNum[counter] + 2
        print(f"Slew on dayObs{dayObs}, seqNum = {seqNum}")
        event = [e for e in events if e.seqNum == seqNum][0]
        start = event.begin
        ggrg = await getGGRGSettings(start)
        tma = await getTMASettings(start)
        print(f"Commanded M1M3 settings = {setting}, from TMA = {ggrg}")
        print (F"Commanded TMA Az settings = {commanded_az_settings}, from TMA = {tma.iloc[0]}")
        print (F"Commanded TMA El settings = {commanded_el_settings}, from TMA = {tma.iloc[1]}")
        counter += 1
        print()

# Now check it on 11-Dec-24

In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()
dayObs = 20241211
blockNum = 178        
events = eventMaker.getEvents(dayObs)
 # The lines below are from the logs

for seqNum in [150, 200, 225, 250]:
    print(f"Slew on dayObs{dayObs}, seqNum = {seqNum}")
    event = [e for e in events if e.seqNum == seqNum][0]
    start = event.begin
    ggrg = await getGGRGSettings(start)
    tma = await getTMASettings(start)
    print(f"Commanded M1M3 settings from TMA = {ggrg}")
    print (F"Commanded TMA Az settings (V,A,J) from TMA = {tma[0]}")
    print (F"Commanded TMA El settings from TMA (V,A,J) = {tma[1]}")
    print()